In [173]:
# data science
import math
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import multiprocessing
# utils, tokenization and preprocessing
from gensim import utils
from random import shuffle
import pymorphy2
from scipy.special import expit
from scipy.stats import logistic
from sklearn.metrics import roc_auc_score

In [2]:
train_df = pd.read_csv("data/train_task1_latest.csv", encoding='utf-8')
test_df = pd.read_csv("data/sdsj_A_test.csv", encoding='utf-8')

In [3]:
train_df.head()

,paragraph_id,question_id,paragraph,question,target
0,1094,46273,"В отличие от рыб, земноводные (амфибии) и прес...",С какого года Русское Царство перешло на летои...,0.0
1,7414,19164,В 1049 году Балдуину V удалось отнять у Герман...,Кто упомянул о его первых разногласиях со Штей...,0.0
2,6744,39767,Стремление достичь предельных значений ёмкости...,Как называется имеющая мировое значение эпоха ...,0.0
3,7300,36318,Первый практически пригодный двухтактный газов...,Что усугублялось из-за международного давления...,0.0
4,7077,41534,Требуя от художника углубленного изучения изоб...,Какой характер носят пророчества Леонардо да В...,0.0


## Data Preprocessing

In [4]:
morph = pymorphy2.MorphAnalyzer()
cores = multiprocessing.cpu_count()
num_partitions = 12

In [10]:
# Probably it is better not to tokenize questions
def tokenize(string):
    tokens = utils.simple_preprocess(string)
    result = []
    for token in tokens:
        s = morph.parse(token)[0].normal_form
        if len(s) > 1:
            result.append(s)
    return result

def parallelize_df(df, func):
    df_split = np.array_split(df, num_partitions)
    pool = multiprocessing.Pool(cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

def tokenize_df(df):
    for col in df.columns:
        if not col.endswith("id"):
            df["tokens"] = df[col].apply(tokenize)
    return df

In [11]:
train_df_p = train_df[["paragraph_id", "paragraph"]].drop_duplicates(subset="paragraph_id")
train_df_q = train_df[["question_id", "question"]].drop_duplicates(subset="question_id")
test_df_p = test_df[["paragraph_id", "paragraph"]].drop_duplicates(subset="paragraph_id")
test_df_q = test_df[["question_id", "question"]].drop_duplicates(subset="question_id")

In [12]:
%time train_df_p = parallelize_df(train_df_p, tokenize_df)
%time train_df_q = parallelize_df(train_df_q, tokenize_df)
%time test_df_p = parallelize_df(test_df_p, tokenize_df)
%time test_df_q = parallelize_df(test_df_q, tokenize_df)

CPU times: user 2.46 s, sys: 620 ms, total: 3.08 s
Wall time: 1min 33s
CPU times: user 1.42 s, sys: 456 ms, total: 1.88 s
Wall time: 40.1 s
CPU times: user 552 ms, sys: 248 ms, total: 800 ms
Wall time: 18.9 s
CPU times: user 2.76 s, sys: 852 ms, total: 3.62 s
Wall time: 1min 21s


In [185]:
train_corpus_p = [TaggedDocument(row["tokens"], ["train_p_" + str(row["paragraph_id"])]) for i, row in train_df_p.iterrows()]
train_corpus_q = [TaggedDocument(row["tokens"], ["train_q_" + str(row["question_id"])]) for i, row in train_df_q.iterrows()]
test_corpus_p = [TaggedDocument(row["tokens"], ["test_p_" + str(row["paragraph_id"])]) for i, row in test_df_p.iterrows()]
test_corpus_q = [TaggedDocument(row["tokens"], ["test_q_" + str(row["question_id"])]) for i, row in test_df_q.iterrows()]

In [186]:
corpus = train_corpus_p + train_corpus_q + test_corpus_p + test_corpus_q
shuffle(corpus)

## Doc2Vec

In [187]:
doc2vec = Doc2Vec(dm=0, dbow_words=1, size=300, window=5, min_count=5, iter=50, workers=cores) 
%time doc2vec.build_vocab(corpus) 

CPU times: user 26.6 s, sys: 696 ms, total: 27.3 s
Wall time: 27.3 s


In [188]:
%time doc2vec.train(corpus, total_examples=doc2vec.corpus_count, epochs=doc2vec.iter)

CPU times: user 4h 6min 28s, sys: 5min 59s, total: 4h 12min 28s
Wall time: 1h 15min 35s


86709013

In [228]:
# doc2vec.save("data/doc2vec")
# doc2vec = Doc2Vec.load("data/doc2vec")

## Testing

In [97]:
# dict for test dataset
id2tokens = {}

for i, row in test_df_p.iterrows():
    _id = "p-" + str(row["paragraph_id"])
    id2tokens[_id] = row["tokens"]

for i, row in test_df_q.iterrows():
    _id = "q-" + str(row["question_id"])
    id2tokens[_id] = row["tokens"]

In [197]:
def label_test_df(df):
    df["prediction"] =  df.apply(lambda row: doc2vec.docvecs.similarity("test_p_" + str(row["paragraph_id"]), "test_q_" + str(row["question_id"])),axis=1)
    return df

def label_train_df(df):
    df["prediction"] =  df.apply(lambda row: doc2vec.docvecs.similarity("train_p_" + str(row["paragraph_id"]), "train_q_" + str(row["question_id"])),axis=1)
    return df

In [199]:
%time train_df = parallelize_df(train_df, label_train_df)

CPU times: user 3.41 s, sys: 2.62 s, total: 6.03 s
Wall time: 46.1 s


In [200]:
train_df.head()

,paragraph_id,question_id,paragraph,question,target,prediction
0,1094,46273,"В отличие от рыб, земноводные (амфибии) и прес...",С какого года Русское Царство перешло на летои...,0.0,0.219650
1,7414,19164,В 1049 году Балдуину V удалось отнять у Герман...,Кто упомянул о его первых разногласиях со Штей...,0.0,0.249833
2,6744,39767,Стремление достичь предельных значений ёмкости...,Как называется имеющая мировое значение эпоха ...,0.0,0.127759
3,7300,36318,Первый практически пригодный двухтактный газов...,Что усугублялось из-за международного давления...,0.0,0.233859
4,7077,41534,Требуя от художника углубленного изучения изоб...,Какой характер носят пророчества Леонардо да В...,0.0,0.409753


In [202]:
roc_auc_score(train_df["target"].tolist(), train_df["prediction"].tolist())

0.96428375083316387

In [198]:
%time test_df = parallelize_df(test_df, label_test_df)

CPU times: user 2.2 s, sys: 1.02 s, total: 3.22 s
Wall time: 23.5 s


In [201]:
test_df.head()

,paragraph_id,question_id,paragraph,question,prediction
0,1361,6463,"Передний мозг сильно развит, это самая большая...",В какой мифологии два ворона Хугин и Мунин шеп...,0.195169
1,1403,34696,Мирмекологи исследуют муравьёв как в лаборатор...,Когда инсайдер покупает или гипотезы чего эвол...,0.331052
2,1435,13751,Волновая: свет представляет собой волну в неви...,"Какие предположения, по мнению Ньютона, допуст...",0.267723
3,341,38544,Живые организмы подчиняются началам термодинам...,В каких условиях метаболизм поддерживает поряд...,0.524390
4,1317,14589,Файлы нелатинского текста в Юникоде всегда зан...,Каким мелодиям Прокофьева особенно не укладыва...,0.439707


In [225]:
test_df[['paragraph_id', 'question_id', 'prediction']].to_csv("data/prediction.csv", index=False)